In [1]:
from shared.models import *
from core.dbs import setup_db
from pydantic import BaseModel

In [2]:
await setup_db()

Connecting to DB Docsense...
Db connected


In [95]:
from datetime import time, timedelta, datetime  # Import time manipulation classes
from typing import List, Optional, Union
from fastapi import HTTPException
from pydantic import BaseModel
from enum import Enum

# Enum representing time slots
class TimeSlot(str, Enum):
    MORNING = "MORNING"
    MIDDAY = "MIDDAY"
    AFTERNOON_EVENING = "AFTERNOON_EVENING"
    NIGHT = "NIGHT"

# Updated TIME_SLOT_MAPPING
TIME_SLOT_MAPPING = {
    TimeSlot.MORNING: (time(9, 0), time(12, 0)),
    TimeSlot.MIDDAY: (time(12, 0), time(15, 0)),
    TimeSlot.AFTERNOON_EVENING: (time(15, 0), time(20, 0)),
    TimeSlot.NIGHT: (time(20, 0), time(23, 0))
}

# Example customizations for sub-services
class CookingCustomization(BaseModel):
    sub_service_id: str
    number_of_people: int

class CleaningCustomization(BaseModel):
    sub_service_id: str
    number_of_rooms: int

# BookingRequest model that accepts multiple sub-services and customizations
class BookingRequest(BaseModel):
    sub_service_ids: List[str]
    booked_date: datetime  # The date on which the booking is made
    provider_id: str
    client_id: str
    time_slot: TimeSlot
    customizations: Optional[List[Union[CookingCustomization, CleaningCustomization]]]

async def book_service(booking_request: BookingRequest):
    # Fetch provider and their existing bookings
    provider = await ServiceProvider.get_document(doc_id=booking_request.provider_id)
    if not provider:
        raise HTTPException(status_code=404, detail="Provider not found")
    
    # Fetch all bookings for the provider within the requested time slot
    provider_bookings = await Booking.search_document({"provider_id": booking_request.provider_id, "deleted_at": None})

    # Get booked time slots within the selected time slot
    provider_booked_time_slots = [
        booking for booking in provider_bookings if booking.time_slot == booking_request.time_slot
    ]

    # Fetch all sub-services to get their durations and base prices
    sub_services = [await SubService.get_document(doc_id=sub_service_id) for sub_service_id in booking_request.sub_service_ids]
    if not sub_services:
        raise HTTPException(status_code=404, detail="One or more sub-services not found")

    # Calculate total duration and total price based on customizations
    total_duration = timedelta()
    total_price = 0

    # Dictionary to store sub-service durations and prices
    sub_service_details = {}
    
    for sub_service in sub_services:
        # Calculate base duration and price
        sub_service_duration = timedelta(hours=sub_service.duration)
        sub_service_price = sub_service.base_price

        # Apply customizations if provided
        for customization in booking_request.customizations or []:
            
            if customization.sub_service_id == str(sub_service.id):
                if isinstance(customization, CookingCustomization):
                    # Increase price based on number of people
                    sub_service_price += (customization.number_of_people -2)  * sub_service.price_per_extra_person   # 10 currency units per person

                elif isinstance(customization, CleaningCustomization):
                    # Increase price based on number of rooms
                    print(sub_service.price_per_extra_room)
                    sub_service_price += (customization.number_of_rooms - 2) * sub_service.price_per_extra_room  # 15 currency units per room

        # Store the sub-service's total duration and price
        sub_service_details[sub_service.id] = {
            "duration": sub_service_duration,
            "price": sub_service_price
        }

        # Add to the total duration and price
        total_duration += sub_service_duration
        total_price += sub_service_price

    # Fetch the start and end time of the requested time slot (morning, midday, etc.)
    slot_start, slot_end = TIME_SLOT_MAPPING[booking_request.time_slot]

    # Calculate available time within the time slot by removing conflicts with booked slots
    available_times = [(slot_start, slot_end)]  # Initially, the entire time slot is available

    # Remove booked times from available times
    for booking in provider_booked_time_slots:
        booked_start, booked_end = booking.start_time.time(), booking.end_time.time()

        for i, (avail_start, avail_end) in enumerate(available_times):
            if avail_start < booked_start < avail_end:  # Conflict at the start
                available_times[i] = (avail_start, booked_start)
            if avail_start < booked_end < avail_end:  # Conflict at the end
                available_times[i] = (booked_end, avail_end)
            if booked_start <= avail_start and booked_end >= avail_end:  # Fully booked
                available_times[i] = None

        # Clean up any None values (fully booked slots)
        available_times = [slot for slot in available_times if slot]

    # Now check if any available time can accommodate the combined duration of all sub-services
    booking_start_time = None
    for avail_start, avail_end in available_times:
        # Combine available start and end times with the booked date to get datetime objects
        avail_start_dt = datetime.combine(booking_request.booked_date, avail_start)
        avail_end_dt = datetime.combine(booking_request.booked_date, avail_end)

        # Check if the available slot can accommodate the total duration
        if avail_end_dt - avail_start_dt >= total_duration:
            booking_start_time = avail_start_dt
            break

    if not booking_start_time:
        raise HTTPException(status_code=400, detail="No available time slots for the selected time period.")

    # Calculate end time based on combined duration
    booking_end_time = booking_start_time + total_duration

    # Create the single booking entry with combined sub-service details
    booking = Booking(
        provider_id=booking_request.provider_id,
        client_id=booking_request.client_id,
        subservice_ids=booking_request.sub_service_ids,  # Storing sub-services with their details
        time_slot=booking_request.time_slot,
        start_time=booking_start_time,
        end_time=booking_end_time,
        frequency=ServiceFrequency.ONE_TIME,
        total_price=total_price,
        booked_date=booking_request.booked_date,
    )

    # Save the booking to the database
    await Booking.save_document(doc=booking)

    return booking


In [100]:
booking_request = BookingRequest(
    sub_service_ids = ["670591e8b3934169a2ddceaa", "67068874e52f7c540001fda7"],
    provider_id =  "67059306b3934169a2ddceab",
    client_id = "dddd",
    time_slot =  TimeSlot.AFTERNOON_EVENING.value,
    booked_date = datetime.now(),
    customizations = [
        CleaningCustomization(sub_service_id = "670591e8b3934169a2ddceaa", number_of_rooms = 3),
    ]
)


In [101]:
await book_service(booking_request)

5.0


Booking(id=ObjectId('67068d25c76efc79fedd3a5e'), revision_id=None, created_at=datetime.datetime(2024, 10, 9, 14, 3, 17, 694777), updated_at=None, deleted_at=None, relations={}, client_id='dddd', provider_id='67059306b3934169a2ddceab', booked_date=datetime.datetime(2024, 10, 9, 14, 3, 17, 429505), subservice_ids=['670591e8b3934169a2ddceaa', '67068874e52f7c540001fda7'], frequency=<ServiceFrequency.ONE_TIME: 'one_time'>, start_time=datetime.datetime(2024, 10, 9, 18, 0), end_time=datetime.datetime(2024, 10, 9, 19, 30), time_slot='AFTERNOON_EVENING', total_price=25.0)

In [ ]:
TimeSlot.MORNING.value

In [5]:
ss = await SubService.get_document(doc_id = "670591e8b3934169a2ddceaa")

In [61]:
provider_bookings = await Booking.search_document({"provider_id": "67059306b3934169a2ddceab"})


In [62]:
provider_bookings[0].deleted_at = None

In [63]:
await Booking.save_document(doc = provider_bookings[0])

Booking(id=ObjectId('670681502278f601358a3175'), revision_id=None, created_at=datetime.datetime(2024, 10, 9, 13, 12, 48, 892000), updated_at=None, deleted_at=None, relations={}, client_id='dddd', provider_id='67059306b3934169a2ddceab', booked_date=None, subservice_ids=['670591e8b3934169a2ddceaa'], frequency=<ServiceFrequency.ONE_TIME: 'one_time'>, start_time=None, end_time=None, time_slot=None, total_price=10.0)

In [64]:
 booking_request

BookingRequest(sub_service_ids=['670591e8b3934169a2ddceaa', '67068874e52f7c540001fda7'], booked_date=datetime.datetime(2024, 10, 9, 13, 56, 59, 736521), provider_id='67059306b3934169a2ddceab', client_id='dddd', time_slot=<TimeSlot.AFTERNOON_EVENING: 'AFTERNOON_EVENING'>, customizations=[CleaningCustomization(sub_service_id='670591e8b3934169a2ddceaa', number_of_rooms=3)])

In [28]:
s = await SubService.save_document(doc = ss)

In [92]:
provider_bookings = await Booking.search_document({"provider_id": "67059306b3934169a2ddceab", "deleted_at": None})


In [93]:
for booking in provider_bookings:
    booking.deleted_at = datetime.now()
    await Booking.save_document(doc = booking)

In [94]:
provider_bookings

[]

In [29]:
s

SubService(id=ObjectId('670591e8b3934169a2ddceaa'), revision_id=None, created_at=datetime.datetime(2024, 10, 8, 20, 11, 4, 728000), updated_at=None, deleted_at=None, relations={}, name=<SubServiceName.DEEP_CLEAN: 'deep_clean'>, parent_service_id='67058f88b3934169a2ddcea9', base_price=10.0, description='DDDD', duration=0.5)

In [32]:
ss = SubService(name = SubServiceName.REGULAR_CLEAN, parent_service_id = "67058f88b3934169a2ddcea9", base_price = 10.0, description = "sss")

In [30]:
await ParentService.search_document({})

[ParentService(id=ObjectId('67058f88b3934169a2ddcea9'), revision_id=None, created_at=datetime.datetime(2024, 10, 8, 20, 0, 57, 709000), updated_at=None, deleted_at=None, relations={}, name='Cleaning', image='s')]

In [35]:
ss = await SubService.save_document(ss)

In [36]:
ss.duration = 1.0

In [37]:
ss = await SubService.save_document(ss)

In [38]:
ss

SubService(id=ObjectId('67068874e52f7c540001fda7'), revision_id=None, created_at=datetime.datetime(2024, 10, 9, 13, 43, 0, 528000), updated_at=None, deleted_at=None, relations={}, name=<SubServiceName.REGULAR_CLEAN: 'regular_clean'>, parent_service_id='67058f88b3934169a2ddcea9', base_price=10.0, description='sss', duration=1.0)

In [8]:
ss = await SubService.get_document(doc_id = "67068874e52f7c540001fda7")

In [9]:
ss.price_per_extra_room = 5
ss = await SubService.save_document(ss)

In [88]:
 customizations = [
        CleaningCustomization(sub_service_id = "670591e8b3934169a2ddceaa", number_of_rooms = 3),
        CleaningCustomization(sub_service_id = "67068874e52f7c540001fda7", number_of_rooms = 4)
    ]